#pip install scipy


In [1]:
import numpy as np
import pandas as pd
import random
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind

### **Experiment 1: Agent scheduling**



In [2]:
random.seed(50)
# Constants
start_time = datetime(2023, 1, 1, 8, 0)
end_time = datetime(2023, 4, 1, 16, 0)
NUMBER_OF_CUSTOMERS = 500
NUMBER_OF_AGENTS = 20

# Base lambda values
lambda_base_weekday = 224/8/NUMBER_OF_CUSTOMERS
lambda_base_weekend = 100/8/NUMBER_OF_CUSTOMERS

# Define customer and agent attributes
customers = [{'id': i+1,
              'lambda_weekday_morning': lambda_base_weekday + np.random.exponential(scale=0.0001),
              'lambda_weekday_afternoon': lambda_base_weekday + np.random.exponential(scale=0.0001),
              'lambda_weekend_morning': lambda_base_weekend + np.random.exponential(scale=0.00001),
              'lambda_weekend_afternoon': lambda_base_weekend + np.random.exponential(scale=0.00001),
              'tolerance': timedelta(minutes=np.random.uniform(2, 8)),
              'next_possible_call_time': start_time} for i in range(NUMBER_OF_CUSTOMERS)]



def truncated_exponential_with_outlier_inverse(lambda_scale, min_val=3*60, max_val=13*60, outlier_val=25*60, outlier_prob=0.05):
    # Check for the chance of an outlier
    if np.random.random() < outlier_prob:
        return outlier_val
    
    # CDF values for the truncation limits
    cdf_min = 1 - np.exp(-min_val / lambda_scale)
    cdf_max = 1 - np.exp(-max_val / lambda_scale)
    
    # Generate a random number between cdf_min and cdf_max
    u = np.random.uniform(cdf_min, cdf_max)
    
    # Apply the inverse CDF to get the sampled value
    sampled_value = -lambda_scale * np.log(1 - u)
    
    return sampled_value


# Function to simulate call generation using a nonhomogeneous Poisson process (NHPP)
def simulate_calls_for_customer_nhpp(customer, start_time, end_time, high_volume_period):
    calls = []
    while True:
        if high_volume_period == "morning":
            if 8 <= start_time.hour < 12:
                lambda_val = 0.6 * (customer['lambda_weekday_morning'] if start_time.weekday() < 6 else customer['lambda_weekend_morning'])
            else:
                lambda_val = 0.4 * (customer['lambda_weekday_afternoon'] if start_time.weekday() < 6 else customer['lambda_weekend_afternoon'])
        elif high_volume_period == "afternoon":
            if 8 <= start_time.hour < 12:
                lambda_val = 0.6 * (customer['lambda_weekday_afternoon'] if start_time.weekday() < 6 else customer['lambda_weekend_afternoon'])
               # lambda_val = 0.4 * (customer['lambda_weekday_morning'] if start_time.weekday() < 6 else customer['lambda_weekend_morning'])
            else:
               # lambda_val = 0.6 * (customer['lambda_weekday_afternoon'] if start_time.weekday() < 6 else customer['lambda_weekend_afternoon'])
                lambda_val = 0.4 * (customer['lambda_weekday_morning'] if start_time.weekday() < 6 else customer['lambda_weekend_morning'])
                
        next_call_time = start_time + timedelta(hours=np.random.exponential(1 / lambda_val))
        if next_call_time <= end_time:
            calls.append({'customer_id': customer['id'], 'call_time': next_call_time, 'tolerance': customer['tolerance']})
            start_time = next_call_time
        else:
            break
    return calls



# Optimized simulation function
def optimized_simulation(run, high_volume_period, agents_split):
    # Initialize lists to store call logs, answered calls, and wait times
    call_logs, answered_calls, wait_times = [], [], []
    # List to store average wait times for each day
    daily_avg_wait_times = []
    # Start time for the simulation
    current_time = start_time

    # Initialize agents for the morning shift based on the agents' split configuration
    agents_morning = [{'id': i+1, 
                       # Set agent responsiveness, higher for peak times if agent split indicates a high volume period in the morning
                       'responsiveness': min(1, 0.97 + np.random.exponential(scale=0.01)) if agents_split == [5,15] else min(1, 0.93 + np.random.exponential(scale=0.01)),   
                       'available_until': start_time,  # When the agent becomes available (start of the day)
                       'talk_duration_lambda': 5*60}  # Average talk duration for the agent (5 minutes here)
                      for i in range(agents_split[0])]  # Create the specified number of morning agents

    # Initialize agents for the afternoon shift based on the agents' split configuration
    agents_afternoon = [{'id': i+1 + agents_split[0],    
                         # Set agent responsiveness, higher for peak times if agent split indicates a high volume period in the afternoon
                         'responsiveness': min(1, 0.97 + np.random.exponential(scale=0.01)) if agents_split == [15,5] else min(1, 0.93 + np.random.exponential(scale=0.01)),
                         'available_until': start_time,  # When the agent becomes available (start of the day)
                         'talk_duration_lambda': 6*60}  # Average talk duration for the agent (6 minutes here)
                        for i in range(agents_split[1])]  # Create the specified number of afternoon agents

    # Main simulation loop
    while current_time <= end_time:
        # Define the end of the current day's working hours
        end_of_day = datetime(current_time.year, current_time.month, current_time.day, 16, 0)
        # Generate call logs for the current day
        call_logs.extend([call for customer in customers for call in simulate_calls_for_customer_nhpp(customer, current_time, end_of_day, high_volume_period)])

        # Determine active agents based on the current time of day
        if 8 <= current_time.hour < 12:
            active_agents = agents_morning
        else:
            active_agents = agents_afternoon        

        # Process each call in the call logs
        for call in sorted(call_logs, key=lambda x: x['call_time']):
            # Find agents that are available to take the call
            available_agents = sorted([agent for agent in active_agents if agent['available_until'] <= call['call_time']], key=lambda x: x['available_until'])
            # If no agents are available or the agent doesn't answer, log the call as abandoned
            if not available_agents or random.random() > available_agents[0]['responsiveness']:
                answered_calls.append({'customer_id': call['customer_id'], 'call_time': call['call_time'], 'status': 'abandoned', 'agent_id': 'NA', 'call_duration': '00:00', 'wait_time': timedelta(seconds=0)})
                continue
            # Select the first available agent to handle the call
            agent = available_agents[0]
            # Calculate the wait time for the call
            wait_time = min(call['tolerance'], call['call_time'] - agent['available_until'])
            # Simulate the call duration
            call_duration_seconds = truncated_exponential_with_outlier_inverse(4*60)
            # Update the agent's next available time based on the call duration
            agent['available_until'] = call['call_time'] + timedelta(seconds=call_duration_seconds)
            # Log the answered call with the relevant details
            answered_calls.append({'customer_id': call['customer_id'], 'call_time': call['call_time'], 'status': 'answered', 'agent_id': agent['id'], 'call_duration': timedelta(seconds=call_duration_seconds), 'wait_time': wait_time})
            # Collect wait time for analysis
            wait_times.append(wait_time)

        # Calculate and store the average wait time for the current day
        if wait_times:  # Check if there are any wait times to avoid division by zero
            daily_avg_wait_time_for_current_day = np.mean(wait_times)
            daily_avg_wait_times.append(daily_avg_wait_time_for_current_day)
            wait_times.clear()  # Clear wait times for the next day
        else:
            daily_avg_wait_times.append(0)

        # Clear call logs for the next day
        call_logs.clear()
        # Move to the next day at the start of working hours
        current_time += timedelta(days=1)
        current_time = datetime(current_time.year, current_time.month, current_time.day, 8, 0)

    # Compute the overall average wait time across all days
    overall_avg_wait_time = np.mean(daily_avg_wait_times)
    # Compute the answer rate for the run
    answer_rate_run = len([call for call in answered_calls if call['status'] == 'answered']) / len(answered_calls)

    # Return the simulation results as a dictionary
    return {
        'average_wait_time': overall_avg_wait_time,
        'answer_rate': answer_rate_run,
        'answered_calls': answered_calls
    }


In [3]:
# Define scenarios
scenarios = {
    "High volume in morning & constant availability": ("morning", [10, 10]),
    "High volume in afternoon & constant availability": ("afternoon", [10, 10]),
    "High volume in morning & higher availability in morning": ("morning", [5, 15]),
    "High volume in afternoon & higher availability in afternoon": ("afternoon", [5, 15])
}

# Number of desired runs for each scenario
num_runs = 1000

# Store results for all scenarios
all_results = {}

for name, (period, split) in scenarios.items():
    results_for_scenario = []
    for run in range(num_runs):
        result = optimized_simulation(run, period, split)
        results_for_scenario.append(result)

        # Print the results for immediate feedback
        print(f"Scenario: {name}, Simulation {run + 1}:")
        print(f"    Average wait time: {result['average_wait_time'].total_seconds()/60:.2f} minutes")
        print(f"    Answer rate: {result['answer_rate']*100:.2f}%")
        print("")

    all_results[name] = results_for_scenario


Scenario: High volume in morning & constant availability, Simulation 1:
    Average wait time: 5.00 minutes
    Answer rate: 93.78%

Scenario: High volume in morning & constant availability, Simulation 2:
    Average wait time: 5.00 minutes
    Answer rate: 93.85%

Scenario: High volume in morning & constant availability, Simulation 3:
    Average wait time: 5.00 minutes
    Answer rate: 93.44%

Scenario: High volume in morning & constant availability, Simulation 4:
    Average wait time: 5.01 minutes
    Answer rate: 93.81%

Scenario: High volume in morning & constant availability, Simulation 5:
    Average wait time: 5.02 minutes
    Answer rate: 93.14%

Scenario: High volume in morning & constant availability, Simulation 6:
    Average wait time: 5.02 minutes
    Answer rate: 94.00%

Scenario: High volume in morning & constant availability, Simulation 7:
    Average wait time: 5.02 minutes
    Answer rate: 94.13%

Scenario: High volume in morning & constant availability, Simulation 

Scenario: High volume in morning & constant availability, Simulation 63:
    Average wait time: 5.00 minutes
    Answer rate: 94.24%

Scenario: High volume in morning & constant availability, Simulation 64:
    Average wait time: 5.06 minutes
    Answer rate: 94.09%

Scenario: High volume in morning & constant availability, Simulation 65:
    Average wait time: 5.03 minutes
    Answer rate: 93.68%

Scenario: High volume in morning & constant availability, Simulation 66:
    Average wait time: 5.00 minutes
    Answer rate: 94.38%

Scenario: High volume in morning & constant availability, Simulation 67:
    Average wait time: 5.01 minutes
    Answer rate: 93.68%

Scenario: High volume in morning & constant availability, Simulation 68:
    Average wait time: 5.00 minutes
    Answer rate: 94.43%

Scenario: High volume in morning & constant availability, Simulation 69:
    Average wait time: 5.01 minutes
    Answer rate: 94.93%

Scenario: High volume in morning & constant availability, Simu

Scenario: High volume in morning & constant availability, Simulation 124:
    Average wait time: 5.02 minutes
    Answer rate: 93.57%

Scenario: High volume in morning & constant availability, Simulation 125:
    Average wait time: 4.99 minutes
    Answer rate: 94.30%

Scenario: High volume in morning & constant availability, Simulation 126:
    Average wait time: 5.04 minutes
    Answer rate: 93.77%

Scenario: High volume in morning & constant availability, Simulation 127:
    Average wait time: 4.99 minutes
    Answer rate: 94.08%

Scenario: High volume in morning & constant availability, Simulation 128:
    Average wait time: 5.03 minutes
    Answer rate: 93.54%

Scenario: High volume in morning & constant availability, Simulation 129:
    Average wait time: 5.00 minutes
    Answer rate: 94.04%

Scenario: High volume in morning & constant availability, Simulation 130:
    Average wait time: 4.98 minutes
    Answer rate: 93.91%

Scenario: High volume in morning & constant availabilit

Scenario: High volume in morning & constant availability, Simulation 185:
    Average wait time: 5.05 minutes
    Answer rate: 93.13%

Scenario: High volume in morning & constant availability, Simulation 186:
    Average wait time: 5.00 minutes
    Answer rate: 94.23%

Scenario: High volume in morning & constant availability, Simulation 187:
    Average wait time: 4.99 minutes
    Answer rate: 93.61%

Scenario: High volume in morning & constant availability, Simulation 188:
    Average wait time: 5.00 minutes
    Answer rate: 93.75%

Scenario: High volume in morning & constant availability, Simulation 189:
    Average wait time: 5.00 minutes
    Answer rate: 93.84%

Scenario: High volume in morning & constant availability, Simulation 190:
    Average wait time: 5.02 minutes
    Answer rate: 94.03%

Scenario: High volume in morning & constant availability, Simulation 191:
    Average wait time: 5.04 minutes
    Answer rate: 93.90%

Scenario: High volume in morning & constant availabilit

Scenario: High volume in morning & constant availability, Simulation 246:
    Average wait time: 5.02 minutes
    Answer rate: 94.26%

Scenario: High volume in morning & constant availability, Simulation 247:
    Average wait time: 5.03 minutes
    Answer rate: 93.64%

Scenario: High volume in morning & constant availability, Simulation 248:
    Average wait time: 5.02 minutes
    Answer rate: 93.86%

Scenario: High volume in morning & constant availability, Simulation 249:
    Average wait time: 5.03 minutes
    Answer rate: 93.65%

Scenario: High volume in morning & constant availability, Simulation 250:
    Average wait time: 4.99 minutes
    Answer rate: 94.14%

Scenario: High volume in morning & constant availability, Simulation 251:
    Average wait time: 4.99 minutes
    Answer rate: 94.20%

Scenario: High volume in morning & constant availability, Simulation 252:
    Average wait time: 5.00 minutes
    Answer rate: 93.87%

Scenario: High volume in morning & constant availabilit

...

Scenario: High volume in afternoon & higher availability in afternoon, Simulation 766:
    Average wait time: 4.58 minutes
    Answer rate: 95.70%

Scenario: High volume in afternoon & higher availability in afternoon, Simulation 767:
    Average wait time: 4.59 minutes
    Answer rate: 94.73%

Scenario: High volume in afternoon & higher availability in afternoon, Simulation 768:
    Average wait time: 4.58 minutes
    Answer rate: 94.94%

Scenario: High volume in afternoon & higher availability in afternoon, Simulation 769:
    Average wait time: 4.55 minutes
    Answer rate: 95.69%

Scenario: High volume in afternoon & higher availability in afternoon, Simulation 770:
    Average wait time: 4.56 minutes
    Answer rate: 95.55%

Scenario: High volume in afternoon & higher availability in afternoon, Simulation 771:
    Average wait time: 4.60 minutes
    Answer rate: 95.10%

Scenario: High volume in afternoon & higher availability in afternoon, Simulation 772:
    Average wait time: 4.

Scenario: High volume in afternoon & higher availability in afternoon, Simulation 822:
    Average wait time: 4.62 minutes
    Answer rate: 95.98%

Scenario: High volume in afternoon & higher availability in afternoon, Simulation 823:
    Average wait time: 4.61 minutes
    Answer rate: 95.08%

Scenario: High volume in afternoon & higher availability in afternoon, Simulation 824:
    Average wait time: 4.57 minutes
    Answer rate: 95.39%

Scenario: High volume in afternoon & higher availability in afternoon, Simulation 825:
    Average wait time: 4.60 minutes
    Answer rate: 94.84%

Scenario: High volume in afternoon & higher availability in afternoon, Simulation 826:
    Average wait time: 4.56 minutes
    Answer rate: 95.52%

Scenario: High volume in afternoon & higher availability in afternoon, Simulation 827:
    Average wait time: 4.61 minutes
    Answer rate: 95.51%

Scenario: High volume in afternoon & higher availability in afternoon, Simulation 828:
    Average wait time: 4.

Scenario: High volume in afternoon & higher availability in afternoon, Simulation 878:
    Average wait time: 4.58 minutes
    Answer rate: 94.35%

Scenario: High volume in afternoon & higher availability in afternoon, Simulation 879:
    Average wait time: 4.58 minutes
    Answer rate: 94.90%

Scenario: High volume in afternoon & higher availability in afternoon, Simulation 880:
    Average wait time: 4.56 minutes
    Answer rate: 94.92%

Scenario: High volume in afternoon & higher availability in afternoon, Simulation 881:
    Average wait time: 4.59 minutes
    Answer rate: 95.59%

Scenario: High volume in afternoon & higher availability in afternoon, Simulation 882:
    Average wait time: 4.57 minutes
    Answer rate: 95.25%

Scenario: High volume in afternoon & higher availability in afternoon, Simulation 883:
    Average wait time: 4.58 minutes
    Answer rate: 94.99%

Scenario: High volume in afternoon & higher availability in afternoon, Simulation 884:
    Average wait time: 4.

Scenario: High volume in afternoon & higher availability in afternoon, Simulation 935:
    Average wait time: 4.59 minutes
    Answer rate: 94.51%

Scenario: High volume in afternoon & higher availability in afternoon, Simulation 936:
    Average wait time: 4.55 minutes
    Answer rate: 94.44%

Scenario: High volume in afternoon & higher availability in afternoon, Simulation 937:
    Average wait time: 4.60 minutes
    Answer rate: 95.24%

Scenario: High volume in afternoon & higher availability in afternoon, Simulation 938:
    Average wait time: 4.59 minutes
    Answer rate: 95.38%

Scenario: High volume in afternoon & higher availability in afternoon, Simulation 939:
    Average wait time: 4.54 minutes
    Answer rate: 95.06%

Scenario: High volume in afternoon & higher availability in afternoon, Simulation 940:
    Average wait time: 4.56 minutes
    Answer rate: 95.08%

Scenario: High volume in afternoon & higher availability in afternoon, Simulation 941:
    Average wait time: 4.

Scenario: High volume in afternoon & higher availability in afternoon, Simulation 991:
    Average wait time: 4.58 minutes
    Answer rate: 95.02%

Scenario: High volume in afternoon & higher availability in afternoon, Simulation 992:
    Average wait time: 4.57 minutes
    Answer rate: 95.70%

Scenario: High volume in afternoon & higher availability in afternoon, Simulation 993:
    Average wait time: 4.60 minutes
    Answer rate: 95.19%

Scenario: High volume in afternoon & higher availability in afternoon, Simulation 994:
    Average wait time: 4.57 minutes
    Answer rate: 94.47%

Scenario: High volume in afternoon & higher availability in afternoon, Simulation 995:
    Average wait time: 4.57 minutes
    Answer rate: 95.23%

Scenario: High volume in afternoon & higher availability in afternoon, Simulation 996:
    Average wait time: 4.52 minutes
    Answer rate: 95.31%

Scenario: High volume in afternoon & higher availability in afternoon, Simulation 997:
    Average wait time: 4.

In [4]:
# Helper function to convert decimal minutes to "minutes:seconds" format
def minutes_to_min_sec(decimal_minutes):
    """
    Convert decimal minutes to "minutes:seconds" format.

    Args:
    - decimal_minutes (float): Time in decimal format.

    Returns:
    - str: Time in "minutes:seconds" format.
    """
    minutes = int(decimal_minutes)  # Extract whole minutes
    seconds = int((decimal_minutes - minutes) * 60)  # Extract remaining seconds
    return f"{minutes}:{seconds:02}"  # Return formatted string

# Analyze the results
analysis = {}

# Iterate through each scenario and its corresponding results
for scenario, results in all_results.items():
    # Extract wait times (in seconds) for all runs
    avg_wait_times = [result['average_wait_time'].total_seconds() for result in results]
    # Extract answer rates for all runs
    avg_answer_rates = [result['answer_rate'] for result in results]

    # Store the average and standard deviation for both metrics in the analysis dictionary
    analysis[scenario] = {
        # Convert average wait time to "minutes:seconds" format
        "Average Wait Time/day": minutes_to_min_sec(np.mean(avg_wait_times)/60),
        # Convert standard deviation of wait time to "minutes:seconds" format
        "Std Dev Wait Time": minutes_to_min_sec(np.std(avg_wait_times)/60),
        # Convert average answer rate to percentage and format as string
        "Average Answer Rate/day": f"{np.mean(avg_answer_rates)*100:.2f}%",
        # Convert standard deviation of answer rate to percentage and format as string
        "Std Dev Answer Rate": f"{np.std(avg_answer_rates)*100:.2f}%"
    }

# Convert the analysis to a pandas DataFrame for a clear tabular representation
import pandas as pd

df = pd.DataFrame(analysis).T
df.head()


,Average Wait Time/day,Std Dev Wait Time,Average Answer Rate/day,Std Dev Answer Rate
High volume in morning & constant availability,5:00,0:01,94.00%,0.37%
High volume in afternoon & constant availability,5:00,0:01,93.98%,0.39%
High volume in morning & higher availability in morning,4:35,0:01,95.06%,0.42%
High volume in afternoon & higher availability in afternoon,4:34,0:01,95.07%,0.40%


##### 